In [8]:
import os
import openai
import gradio as gr

# OpenAI API 키 설정
openai.api_key = os.getenv("OPENAI_API_KEY")
if openai.api_key is None:
    raise ValueError("❌ OPENAI_API_KEY 환경변수가 설정되지 않았습니다.")

# Gradio interface 함수
def respond(user_message, chat_history):
    # 기존 채팅 기록에 사용자 메시지 추가
    chat_history.append((user_message, ""))
    messages = []
    
    # 채팅 기록을 OpenAI API 형식으로 변환
    for user, assistant in chat_history:
        messages.append({"role": "user", "content": user})
        if assistant:  # 빈 응답이 아닌 경우만 추가
            messages.append({"role": "assistant", "content": assistant})
    
    # 응답 생성 및 스트리밍
    bot_response = ""
    for chunk in openai.chat.completions.create(
        model="gpt-4",
        messages=messages,
        stream=True,
    ):
        if chunk.choices[0].delta.content:
            bot_response += chunk.choices[0].delta.content
            chat_history[-1] = (user_message, bot_response)
            yield chat_history

# Gradio 인터페이스 설정
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="메시지를 입력하세요...", show_label=False, lines=1)
    clear = gr.Button("대화 초기화")
    
    msg.submit(respond, [msg, chatbot], chatbot)
    msg.submit(lambda: "", None, msg, queue=False)  # 입력 필드 비우기
    clear.click(lambda: [], None, chatbot, queue=False)  # 대화 초기화

if __name__ == "__main__":
    demo.launch()

/tmp/ipykernel_66899/3298509608.py:36: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.
